# Evergreen Farmers Market Attendance Forecast

In [1]:
%pip install numpy pandas requests
import pandas as pd
from datetime import datetime, date

%pip install openmeteo_requests requests-cache retry-requests
import openmeteo_requests
import requests_cache
from retry_requests import retry

%pip install joblib
import joblib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Create future data

In [2]:
# Must be within future 7 days
future_date = date(2025, 4, 19)

visits_last_week = 2558
visits_two_weeks_ago = 2567

In [3]:
# Evergreem market coorindates
LAT = '43.6843353'
LONG = '-79.367293'

def date_string_from(date):
    return date.strftime("%Y-%m-%d")

def date_from(date_str):
    return datetime.strptime(date_str, "%Y-%m-%d")

# WMO weather codes
# https://www.nodc.noaa.gov/archive/arc0021/0002199/1.1/data/0-data/HTML/WMO-CODE/WMO4677.HTM
def is_sunny(weather_code):
    return weather_code <= 1

def is_cloudy(weather_code):
    return (weather_code >= 2 and weather_code <= 3) or (weather_code >= 45 and weather_code <= 48)

def is_rainy(weather_code):
    return (weather_code >= 51 and weather_code <= 67) or (weather_code >= 80 and weather_code <= 82) or (weather_code >= 95 and weather_code <= 99)

def is_snowy(weather_code):
    return (weather_code >= 71 and weather_code <= 77) or (weather_code >= 85 and weather_code <= 85)

def fetch_weather_forecast_7d():
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": LAT,
        "longitude": LONG,
        "daily": ["weather_code", "temperature_2m_max", "temperature_2m_min"],
        "timezone": "America/New_York"
    }
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]

    # Process daily data. The order of variables needs to be the same as requested.
    daily = response.Daily()
    daily_weather_code = daily.Variables(0).ValuesAsNumpy()
    daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
    daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()

    daily_data = {"date": pd.date_range(
        start = date_string_from(pd.to_datetime(daily.Time(), unit = "s", utc = True)),
        end = date_string_from(pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True)),
        freq = pd.Timedelta(seconds = daily.Interval()),
        inclusive = "left"
    )}

    daily_data["weather_code"] = daily_weather_code
    daily_data["temperature_2m_max"] = daily_temperature_2m_max
    daily_data["temperature_2m_min"] = daily_temperature_2m_min

    return pd.DataFrame(data = daily_data)

weather_forecast = fetch_weather_forecast_7d()
weather_forecast_daily = weather_forecast.loc[weather_forecast['date'] == future_date.strftime("%Y-%m-%d")]
if(weather_forecast_daily.empty):
    print('can\'t find weather forecast')
    exit(1)

weather_code = weather_forecast_daily['weather_code'].iloc[0]

future_data = {
    "ds": [future_date],
    "visits_last_week": [visits_last_week], 
    "visits_two_weeks_ago": [visits_two_weeks_ago],  
    'min_temp': [float(weather_forecast_daily['temperature_2m_min'].iloc[0])],
	'max_temp': [float(weather_forecast_daily['temperature_2m_max'].iloc[0])],
	'is_sunny': [int(is_sunny(weather_code))],
	'is_cloudy': [int(is_cloudy(weather_code))],
	'is_rainy': [int(is_rainy(weather_code))],
	'is_snowy': [int(is_snowy(weather_code))],
}

future_df = pd.DataFrame(future_data)
future_df.head()


,ds,visits_last_week,visits_two_weeks_ago,min_temp,max_temp,is_sunny,is_cloudy,is_rainy,is_snowy
0,2025-04-19,2558,2567,5.55,18.25,0,0,1,0


## Make Prediction

In [4]:
OUTPUT_DIR = 'output'

model = joblib.load(f'{OUTPUT_DIR}/attendance_model.pkl')
scaler = joblib.load(f'{OUTPUT_DIR}/attendance_scaler.pkl')

future_df[['visits_last_week', 'visits_two_weeks_ago', 'min_temp', 'max_temp']] = scaler.transform(future_df[['visits_last_week', 'visits_two_weeks_ago', 'min_temp', 'max_temp']])
future_df.head()

/Users/fangchenhuang/code/evergreen-market-attendance-forecast/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


,ds,visits_last_week,visits_two_weeks_ago,min_temp,max_temp,is_sunny,is_cloudy,is_rainy,is_snowy
0,2025-04-19,-0.640044,-0.584164,-0.019781,0.49278,0,0,1,0


In [5]:
forecast = model.predict(future_df)
forecast["yhat"]

0    2527.455504
Name: yhat, dtype: float64